**Business Problem :** Segmentation and developing marketing strategy for an E-commerce company

## Variables
## InvoiceNo: Invoice number.Abnormal number for each transaction, i.e. invoice. Canceled operation starting with C
## StockCode: Product code. Unique number for each product
## Description: product name
## Quantity: Number of products. It indicates how many of the products on the invoices were sold.
## InvoiceDate: Invoice Date
## UnitPrice: Product Price (Sterlin)
## CustomerID: Unique customer id
## Country: Country of the customer

# CLTV = (Customer_Value / Churn_Rate) x Profit_margin.
# Customer_Value = Average_Order_Value * Purchase_Frequency

In [19]:
#Data Understanding
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [20]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()

(525461, 8)

In [22]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [23]:
df.shape

(525461, 8)

In [24]:
#Data Preparation

df.isnull().sum()


Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [25]:
df.dropna(inplace=True)

In [26]:
df.shape

(417534, 8)

In [30]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [27]:
#removing refunds
df = df[~df["Invoice"].str.contains("C", na=False)]

# Calculating RFM Metrics
# Recency : Time since customer's last purchase
# Frequency : Total number of purchases
# Monetary : Total spending by the customer

In [28]:
df["InvoiceDate"].max()
today_date = dt.datetime(2010, 12, 11)

In [31]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [32]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00000,165,11,372.86000
12347.00000,3,2,1323.32000
12348.00000,74,1,222.16000
12349.00000,43,3,2671.14000
12351.00000,11,1,300.93000


In [33]:
rfm.columns = ['recency', 'frequency', 'monetary']

In [34]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.00000,91.26982,96.94430,1.00000,18.00000,53.00000,136.00000,374.00000
frequency,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
monetary,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000


In [35]:
rfm = rfm[rfm["monetary"] > 0]

In [36]:
#Calculating RFM Scores
## Recency
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

In [37]:
#Frequency
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [ ]:
#Monetary
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

In [42]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

In [44]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,RFM_SCORE
Customer ID,,,,,,
12346.00000,165,11,372.86000,2,5,25
12347.00000,3,2,1323.32000,5,2,52
12348.00000,74,1,222.16000,2,1,21
12349.00000,43,3,2671.14000,3,3,33
12351.00000,11,1,300.93000,5,1,51


In [39]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.00000,91.17254,96.86146,1.00000,18.00000,53.00000,136.00000,374.00000
frequency,4312.00000,4.45571,8.17021,1.00000,1.00000,2.00000,5.00000,205.00000
monetary,4312.00000,2048.23824,8914.48128,2.95000,307.98750,706.02000,1723.14250,349164.35000


In [45]:
#Creating & Analysing RFM Segments
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}


In [46]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)  # birleştirilen skorlar seg_map ile değiştirildi
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,RFM_SCORE,segment
Customer ID,,,,,,,
12346.00000,165,11,372.86000,2,5,25,cant_loose
12347.00000,3,2,1323.32000,5,2,52,potential_loyalists
12348.00000,74,1,222.16000,2,1,21,hibernating
12349.00000,43,3,2671.14000,3,3,33,need_attention
12351.00000,11,1,300.93000,5,1,51,new_customers


In [47]:
# Group RFM mean and frequency values by segments
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])
rfm[rfm["segment"] == "need_attention"].head()

,recency,frequency,monetary,recency_score,frequency_score,RFM_SCORE,segment
Customer ID,,,,,,,
12349.00000,43,3,2671.14000,3,3,33,need_attention
12369.00000,49,3,1791.15000,3,3,33,need_attention
12371.00000,45,3,2179.42000,3,3,33,need_attention
12374.00000,57,3,2246.29000,3,3,33,need_attention
12389.00000,38,3,1433.33000,3,3,33,need_attention


In [48]:
rfm[rfm["segment"] == "new_customers"].index

Float64Index([12351.0, 12385.0, 12386.0, 12427.0, 12441.0, 12538.0, 12686.0,
              12738.0, 12763.0, 12767.0, 12942.0, 12947.0, 12961.0, 12996.0,
              13011.0, 13030.0, 13254.0, 13270.0, 13369.0, 13370.0, 13626.0,
              13955.0, 14414.0, 14491.0, 14576.0, 14589.0, 14594.0, 14627.0,
              14721.0, 14775.0, 15018.0, 15020.0, 15048.0, 15165.0, 15211.0,
              15212.0, 15404.0, 15495.0, 15540.0, 15545.0, 15583.0, 15828.0,
              15880.0, 15894.0, 15899.0, 15919.0, 15922.0, 15923.0, 15939.0,
              15961.0],
             dtype='float64', name='Customer ID')

In [49]:
new_df = pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index
new_df.head()

,new_customer_id
0,12351.00000
1,12385.00000
2,12386.00000
3,12427.00000
4,12441.00000
